In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
spark = (
    SparkSession.builder
        .appName("BronzeToSilver")
        .config("spark.jars.packages",
                "io.delta:delta-spark_2.12:3.2.1")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
        .getOrCreate()
)


bronze_path = "hdfs://master:8020/delta/bronze/movies"
df_bronze = spark.read.format("delta").load(bronze_path)


df_silver = (
    df_bronze
    .dropna(subset=["Title", "Revenue_Millions", "Runtime_Minutes"]) 
    .withColumn("Revenue_Millions", col("Revenue_Millions").cast("double"))
    .withColumn("Runtime_Minutes", col("Runtime_Minutes").cast("int"))
    .withColumn("Metascore", col("Metascore").cast("int"))
    .withColumn("Votes", col("Votes").cast("int"))
)


silver_path = "hdfs://master:8020/delta/silver/movies"
df_silver.write.format("delta") \
    .mode("overwrite") \
    .save(silver_path)

print(f" Silver Layer saved to {silver_path}")


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jupyter/.ivy2/cache
The jars for the packages stored in: /home/jupyter/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a7632463-9a6e-4fec-9ce9-af6dc6e09a4c;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.2.1 in central
	found io.delta#delta-storage;3.2.1 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 178ms :: artifacts dl 7ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.2.1 from central in [default]
	io.delta#delta-storage;3.2.1 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   

 Silver Layer saved to hdfs://master:8020/delta/silver/movies
